# Neural networks for Classification - FashionMNIST

In [ ]:
# Install latest Tensorflow build
!pip install tensorflow
from tensorflow import summary
%load_ext tensorboard

import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

train_set = torchvision.datasets.FashionMNIST(root = ".", train=True, download=True, transform=transforms.ToTensor())
test_set = torchvision.datasets.FashionMNIST(root = ".", train=False, download=True, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False)
torch.manual_seed(0)
# If you are using CuDNN , otherwise you can just ignore
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

In [ ]:
input_data, label = next(iter(train_loader))
plt.imshow(input_data[0,:,:,:].numpy().reshape(28,28), cmap="gray_r");
print("Label is: {}".format(label[0]))
print("Dimension of input data: {}".format(input_data.size()))
print("Dimension of labels: {}".format(label.size()))

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear) or isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)

# CNN implementation

class MyCNN(nn.Module):
  def __init__(self):
    super(MyCNN, self).__init__()
    self.cnn_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5), nn.ReLU(), nn.MaxPool2d(2), nn.Conv2d(32, 64, kernel_size=5), nn.ReLU(), nn.MaxPool2d(2))
    # self.fc_model = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024,256), nn.ReLU(), nn.Linear(256, 10))
    self.fc_model = nn.Sequential(nn.Linear(1024, 1024), nn.ReLU(), nn.Linear(1024,256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 10))

    # self.cnn_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5), nn.Tanh(), nn.MaxPool2d(2), nn.Conv2d(32, 64, kernel_size=5), nn.Tanh(), nn.MaxPool2d(2))
    # self.fc_model = nn.Sequential(nn.Linear(1024, 1024), nn.Tanh(), nn.Linear(1024,256), nn.Tanh(), nn.Linear(256, 10))

    # self.cnn_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5), nn.Sigmoid(), nn.MaxPool2d(2), nn.Conv2d(32, 64, kernel_size=5), nn.Sigmoid(), nn.MaxPool2d(2))
    # self.fc_model = nn.Sequential(nn.Linear(1024, 1024), nn.Sigmoid(), nn.Linear(1024,256), nn.Sigmoid(), nn.Linear(256, 10))

    # self.cnn_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=5), nn.ELU(), nn.MaxPool2d(2), nn.Conv2d(32, 64, kernel_size=5), nn.ELU(), nn.MaxPool2d(2))
    # self.fc_model = nn.Sequential(nn.Linear(1024, 1024), nn.ELU(), nn.Linear(1024,256), nn.ELU(), nn.Linear(256, 10))


  def forward(self, x):
    x = self.cnn_model(x)
    x = x.view(x.size(0), -1)
    x = self.fc_model(x)
    return x

device = torch.device("cuda:0")

def evaluation(dataloader, loss_fn):
  total, correct = 0,0
  val_loss = 0
  net.eval()
  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum().item()
    loss = loss_fn(outputs, labels)
    val_loss += loss.item()

  return val_loss, 100 * correct / total

# for alpha in [0.001, 0.1, 0.5, 1, 10]:
for alpha in [0.1]:

    print(alpha)

    net = MyCNN().to(device)
    net.apply(init_weights)
    loss_fn = nn.CrossEntropyLoss()
    loss_fn.to(device)
    opt = torch.optim.SGD(list(net.parameters()), lr=alpha)

    loss_epoch_array = []
    max_epochs = 30
    loss_epoch = 0
    train_accuracy = []
    valid_accuracy = []
    for epoch in range(max_epochs):
      loss_epoch = 0
      for i, data in enumerate(train_loader, 0):
        net.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        opt.zero_grad()
        outputs = net(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        loss_epoch += loss.item()
      loss_epoch_array.append(loss_epoch)
      _, train_acc = evaluation(train_loader, loss_fn)
      train_accuracy.append(train_acc)
      val_loss, val_acc = evaluation(test_loader, loss_fn)
      valid_accuracy.append(val_acc)
      print(epoch + 1, loss_epoch_array[-1], train_accuracy[-1], val_loss, valid_accuracy[-1])